# Setting up and Getting the Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install patool
! pip install -Uqq fastbook
from fastai.vision.all import *
from fastai.text.all import *
from fastai.collab import *
from fastai.tabular.all import *
from fastai.metrics import error_rate, accuracy
import patoolib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.2 MB/s eta 0:00:00


In [3]:
patoolib.extract_archive("/content/drive/MyDrive/Datasets/WBC_Classification/data.rar", outdir="/content/data")

INFO patool: Extracting /content/drive/MyDrive/Datasets/WBC_Classification/data.rar ...
INFO:patool:Extracting /content/drive/MyDrive/Datasets/WBC_Classification/data.rar ...
INFO patool: ... creating output directory `/content/data'.
INFO:patool:... creating output directory `/content/data'.
INFO patool: running /usr/bin/unrar x -- /content/drive/MyDrive/Datasets/WBC_Classification/data.rar
INFO:patool:running /usr/bin/unrar x -- /content/drive/MyDrive/Datasets/WBC_Classification/data.rar
INFO patool:     with cwd='/content/data', input=''
INFO:patool:    with cwd='/content/data', input=''
INFO patool: ... /content/drive/MyDrive/Datasets/WBC_Classification/data.rar extracted to `/content/data'.
INFO:patool:... /content/drive/MyDrive/Datasets/WBC_Classification/data.rar extracted to `/content/data'.


'/content/data'

In [4]:
path = Path("/content/data/data")

In [5]:
wbc = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    splitter =  RandomSplitter(valid_pct=0.25, seed=42),
    get_y = parent_label,
    item_tfms = RandomResizedCrop(224, min_scale = 0.5),
    batch_tfms = aug_transforms()
)
dls = wbc.dataloaders(path)

# Testing different Models:

## **ResNet101**

In [6]:
from torchvision.models import ResNet101_Weights
metrics=[error_rate, accuracy]
learn_resnet_101 = vision_learner(dls, models.resnet101, pretrained=True, weights=ResNet101_Weights.IMAGENET1K_V1, metrics = metrics)

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 141MB/s]


In [7]:
learn_resnet_101.fine_tune(5)

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.464825,0.196129,0.055127,0.944873,02:48


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.127191,0.143914,0.028666,0.971334,03:44
1,0.092909,0.097201,0.027288,0.972712,03:43
2,0.077915,0.070763,0.016814,0.983186,03:43
3,0.052046,0.045403,0.013230,0.986770,03:44
4,0.020639,0.049608,0.012404,0.987596,03:42


In [8]:
interp_resnet_101=ClassificationInterpretation.from_learner(learn_resnet_101)

In [9]:
interp_resnet_101.print_classification_report()

              precision    recall  f1-score   support

    Basophil       0.99      1.00      0.99        72
  Eosinophil       1.00      0.94      0.97       271
  Lymphocyte       0.98      0.99      0.98       842
    Monocyte       0.96      0.94      0.95       184
  Neutrophil       0.99      1.00      0.99      2259

    accuracy                           0.99      3628
   macro avg       0.98      0.97      0.98      3628
weighted avg       0.99      0.99      0.99      3628



In [10]:
learn_resnet_101.export("learn_resnet_101_export.pkl")

## **ResNet50**

In [11]:
from torchvision.models import ResNet50_Weights
metrics=[error_rate, accuracy]
learn_resnet_50 = vision_learner(dls, models.resnet50, pretrained=True, weights=ResNet50_Weights.IMAGENET1K_V1, metrics = metrics)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 42.6MB/s]


In [12]:
learn_resnet_50.fine_tune(5)

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.600082,0.287704,0.077729,0.922271,01:51


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.151171,0.132278,0.027563,0.972437,02:21
1,0.101878,0.104512,0.021775,0.978225,02:23
2,0.065194,0.073911,0.015987,0.984013,02:22
3,0.039896,0.065418,0.014333,0.985667,02:22
4,0.026261,0.059305,0.013230,0.986770,02:22


In [13]:
interp_resnet_50=ClassificationInterpretation.from_learner(learn_resnet_50)

In [14]:
interp_resnet_50.print_classification_report()

              precision    recall  f1-score   support

    Basophil       1.00      1.00      1.00        72
  Eosinophil       0.99      0.95      0.97       271
  Lymphocyte       0.99      0.98      0.98       842
    Monocyte       0.93      0.95      0.94       184
  Neutrophil       0.99      1.00      0.99      2259

    accuracy                           0.99      3628
   macro avg       0.98      0.98      0.98      3628
weighted avg       0.99      0.99      0.99      3628



In [15]:
learn_resnet_50.export("learn_resnet_50_export.pkl")

## **Inception-V3**

In [ ]:
from torchvision.models import Inception_V3_Weights

metrics = [error_rate, accuracy]

# Load the Inception_V3 body
inception_v3 = models.inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1).eval()
body = create_body(inception_v3, 3, pretrained=True)

# Create a custom head
nf = inception_v3.fc.in_features
head = create_head(nf, dls.c, ps=[0.5], bn_final=False)

# Create the learner with the custom head
learn_inception_v3 = vision_learner(dls, body, custom_head=head, metrics=metrics)

In [ ]:
learn_inception_v3.fine_tune(10)

In [ ]:
interp_inception_v3 = ClassificationInterpretation.from_learner(learn_inception_v3)

In [ ]:
interp_inception_v3.print_classification_report()

In [ ]:
learn_inception_v3.export("learn_inception_v3_export.pkl")

## **ConvNeXt-Small**

In [16]:
from torchvision.models import ConvNeXt_Small_Weights
metrics=[error_rate, accuracy]
learn_convnext_small = vision_learner(dls, models.convnext_small, pretrained=True, weights=ConvNeXt_Small_Weights.IMAGENET1K_V1, metrics = metrics)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Small_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/convnext_small-0c510722.pth" to /root/.cache/torch/hub/checkpoints/convnext_small-0c510722.pth
100%|██████████| 192M/192M [00:03<00:00, 62.6MB/s]


In [17]:
learn_convnext_small.fine_tune(10)

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.654707,0.166248,0.051268,0.948732,03:19


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.221513,0.096759,0.027288,0.972712,05:36
1,0.156602,0.079586,0.022878,0.977122,05:38
2,0.106637,0.063989,0.018467,0.981533,05:37
3,0.080231,0.057766,0.015160,0.984840,05:36
4,0.062054,0.054966,0.013506,0.986494,05:39
5,0.054767,0.051823,0.013506,0.986494,05:36
6,0.050841,0.048812,0.012404,0.987596,05:36
7,0.047058,0.048983,0.012679,0.987321,05:37
8,0.036308,0.048907,0.012679,0.987321,05:37
9,0.036201,0.048721,0.012679,0.987321,05:37


In [18]:
interp_convnext_small = ClassificationInterpretation.from_learner(learn_convnext_small)

In [19]:
interp_convnext_small.print_classification_report()

              precision    recall  f1-score   support

    Basophil       1.00      1.00      1.00        72
  Eosinophil       0.99      0.96      0.97       271
  Lymphocyte       0.98      0.98      0.98       842
    Monocyte       0.94      0.93      0.94       184
  Neutrophil       0.99      1.00      0.99      2259

    accuracy                           0.99      3628
   macro avg       0.98      0.97      0.98      3628
weighted avg       0.99      0.99      0.99      3628



In [21]:
learn_convnext_small.export("learn_convnext_base_export.pkl")